<a href="https://www.kaggle.com/code/davidhalim2004/data-cleaning?scriptVersionId=280113860" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Library importing

In [1]:
!pip install xgboost
import pandas as pd
import joblib
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import gc

# data pre-proccessing

## data reading

In [2]:
chunks = pd.read_csv("/kaggle/input/depi-dataset/data.csv", chunksize=5_000_000)
final_data = pd.concat(chunks, ignore_index=True)

In [3]:
del chunks

In [4]:
preprocessor=joblib.load("/kaggle/input/depi-dataset/pre-proccess.pkl")

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator OrdinalEncoder from version 1.7.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.7.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Pipeline from version 1.7.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info 

## Data spliting

In [5]:
store_models = {}

stores = final_data["store_id"].unique()

for store in stores:
    print(f"\n==== Training model for store: {store} ====")

    # 1️⃣ Filter the data of that store only
    df_store = final_data[final_data["store_id"] == store].copy()

    # 2️⃣ Split into X, y
    X = df_store.drop(columns=["sales"])
    y = df_store["sales"]
    X=preprocessor.fit_transform(X)

    # 3️⃣ Random Split for THIS store ONLY
    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y, test_size=0.20, random_state=42
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=0.15, random_state=42
    )

    # 4️⃣ Fit model
    model = XGBRegressor(
    objective='reg:tweedie',
    tweedie_variance_power=1.35,
    booster='gbtree',
    device='cuda',
    tree_method='hist',
    random_state=42,
    eval_metric='mae',
    learning_rate=0.03,
    max_depth=6,
    min_child_weight=15,
    subsample=0.7,
    colsample_bytree=0.7,
    reg_lambda=6,
    reg_alpha=2,
    n_estimators=3500,
    early_stopping_rounds=40,
    gamma= 1,
    max_delta_step = 1
    )
    
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose = False
    )
    model.score(X_train,y_train) , model.score(X_test,y_test)
    from sklearn.metrics import mean_absolute_error, r2_score

    train_mae = mean_absolute_error(y_train, model.predict(X_train))
    test_mae  = mean_absolute_error(y_test,  model.predict(X_test))
    train_r2  = r2_score(y_train, model.predict(X_train))
    test_r2   = r2_score(y_test,  model.predict(X_test))

    print(f"MAE train: {train_mae:.4f}, test: {test_mae:.4f}")
    print(f"R2  train: {train_r2:.4f}, test: {test_r2:.4f}")
   
    # 5️⃣ Save model
    store_models[store] = model
    print(f"Model for {store} trained and stored.")

    del X, y, X_temp, X_test, X_val, X_train, y_train, y_temp, y_test
    gc.collect()


==== Training model for store: CA_1 ====


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [22:16:01] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


MAE train: 0.4487, test: 0.4601
R2  train: 0.8951, test: 0.8655
Model for CA_1 trained and stored.

==== Training model for store: CA_2 ====
MAE train: 0.3590, test: 0.3680
R2  train: 0.8510, test: 0.8240
Model for CA_2 trained and stored.

==== Training model for store: CA_3 ====
MAE train: 0.6177, test: 0.6324
R2  train: 0.9091, test: 0.8963
Model for CA_3 trained and stored.

==== Training model for store: CA_4 ====
MAE train: 0.2630, test: 0.2703
R2  train: 0.8514, test: 0.8221
Model for CA_4 trained and stored.

==== Training model for store: TX_1 ====
MAE train: 0.3370, test: 0.3487
R2  train: 0.8966, test: 0.8404
Model for TX_1 trained and stored.

==== Training model for store: TX_2 ====
MAE train: 0.4124, test: 0.4236
R2  train: 0.9176, test: 0.8962
Model for TX_2 trained and stored.

==== Training model for store: TX_3 ====
MAE train: 0.3586, test: 0.3697
R2  train: 0.9099, test: 0.8878
Model for TX_3 trained and stored.

==== Training model for store: WI_1 ====
MAE train: 0.

In [6]:
import numpy as np
for store in stores:
    # 1️⃣ Filter the data of that store only
    df_store = final_data[final_data["store_id"] == store].copy()

    # 2️⃣ Split into X, y
    X = df_store.drop(columns=["sales"])
    y = df_store["sales"]
    X=preprocessor.fit_transform(X)

    # 3️⃣ Random Split for THIS store ONLY
    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y, test_size=0.20, random_state=42
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=0.15, random_state=42
    )
    y_pred = store_models[store].predict(X_test)
    # Round predictions and ground truth
    y_pred_rounded = np.round(y_pred).astype(int)
    y_test_rounded = np.round(y_test).astype(int)

    print("\n=== Sample Predictions vs Actuals ===\n")
    for i in range(min(25, len(y_test))):
        print(f"Pred: {y_pred_rounded[i]:>3}   Actual: {y_test_rounded.iloc[i]:>3}")


=== Sample Predictions vs Actuals ===

Pred:   2   Actual:   3
Pred:   2   Actual:   2
Pred:  11   Actual:   8
Pred:   3   Actual:   3
Pred:   0   Actual:   0
Pred:   0   Actual:   0
Pred:   0   Actual:   0
Pred:   0   Actual:   0
Pred:   2   Actual:   2
Pred:   6   Actual:   6
Pred:   0   Actual:   0
Pred:   0   Actual:   0
Pred:   0   Actual:   0
Pred:   2   Actual:   2
Pred:   0   Actual:   0
Pred:   0   Actual:   0
Pred:   0   Actual:   0
Pred:   0   Actual:   0
Pred:   3   Actual:   3
Pred:   6   Actual:   6
Pred:   0   Actual:   0
Pred:   0   Actual:   0
Pred:   2   Actual:   2
Pred:   0   Actual:   0
Pred:   0   Actual:   0

=== Sample Predictions vs Actuals ===

Pred:   0   Actual:   0
Pred:   3   Actual:   3
Pred:  15   Actual:   6
Pred:   2   Actual:   0
Pred:   1   Actual:   1
Pred:   2   Actual:   3
Pred:   0   Actual:   0
Pred:   1   Actual:   1
Pred:   2   Actual:   1
Pred:   1   Actual:   1
Pred:   0   Actual:   0
Pred:   0   Actual:   0
Pred:   0   Actual:   0
Pred:   

In [7]:
for i in store_models.keys():
    joblib.dump(store_models[i], f"/kaggle/working/{i}.pkl")